Allmän inloggning via Windows Authentication.

In [10]:
from sqlalchemy import create_engine, text
import pandas as pd

server = "HP"
database = "EmilEdman"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

with engine.connect() as connection:
    print(f"Anslutning till databasen '{database}' på servern '{server}' lyckades.")

Anslutning till databasen 'EmilEdman' på servern 'HP' lyckades.


Ej i bruk...
Anpassad inloggning för den begränsade usern jag skapade med tillhörande password: zxc-230y

In [ ]:
from sqlalchemy import create_engine, text
from getpass import getpass
from sqlalchemy.exc import OperationalError
import pandas as pd

användarnamn = 'Raffe'
PWD = getpass("Password")
server = 'HP'
database = 'EmilEdman'
driver = 'ODBC Driver 17 for SQL Server'

engine = create_engine(
    f"mssql+pyodbc://{användarnamn}:{PWD}@{server}/{database}?driver={driver}&Encrypt=no")

try:
    with engine.connect() as connection:
        print(f"Anslutning till databasen '{database}' på servern '{server}' lyckades.")
except OperationalError as e:
    print("Misslyckades ansluta till databasen.")
    print(f"Felmeddelande {e}")

Nedan kod är baserad på det vi gjort på lektioner plus query. Jag har tagit LLM till hjälp då jag stötte på flera problem med kontakten mot databasen och outputen.  

In [8]:
# Fråga användaren efter en titel
search_string = input("Ange en boktitel att söka efter: ")

print(f"Söker efter titlar som matchar: {search_string}\n")

# Säker SQL-fråga med parametrisering för att undvika SQL-injektion
query = f"""
WITH UnikaGenrer AS (
    SELECT DISTINCT B.ISBN13, G.Genre
    FROM Böcker B
    JOIN BokGenre BG ON B.ISBN13 = BG.ISBN13
    JOIN Genre G ON BG.GenreId = G.GenreId
)
SELECT TOP 10
    B.ISBN13 AS ISBN, 
    B.Titel, 
    CONCAT(F.Förnamn, ' ', F.Efternamn) AS Författare,
    (SELECT STRING_AGG(Genre, ', ') FROM UnikaGenrer UG WHERE UG.ISBN13 = B.ISBN13) AS Genrer,
    MAX(CASE WHEN BU.Butiksnamn = 'BookStore' THEN L.Saldo END) AS [Saldo BookStore],
    MAX(CASE WHEN BU.Butiksnamn = 'BokAffären' THEN L.Saldo END) AS [Saldo BokAffären],
    MAX(CASE WHEN BU.Butiksnamn = 'Affären med böcker' THEN L.Saldo END) AS [Saldo Affären med böcker]
FROM Böcker B
JOIN BöckerOchFörfattare BOF ON B.ISBN13 = BOF.BokId
JOIN Författare F ON BOF.FörfattareId = F.ID
JOIN LagerSaldo L ON B.ISBN13 = L.ISBN
JOIN Butiker BU ON L.ButiksId = BU.ID
WHERE B.Titel LIKE '%{search_string}%'
GROUP BY B.ISBN13, B.Titel, F.Förnamn, F.Efternamn;
"""

with engine.connect() as conn:
    result = conn.execute(text(query), {"search_string": search_string})
    rows = result.fetchall()

    if not rows:
        print("Inga böcker matchade din sökning")
    else:
        print(f"{'ISBN'.ljust(15)}{'Titel'.ljust(30)}{'Författare'.ljust(30)}{'Genre'.ljust(30)}"
            f"{'Saldo BookStore'.ljust(20)}{'Saldo BokAffären'.ljust(20)}{'Saldo Affären med böcker'.ljust(20)}")

        for row in rows:
            print(f"{str(row[0]).ljust(15)}{str(row[1]).ljust(30)}{str(row[2]).ljust(30)}{str(row[3]).ljust(30)}"
                f"{str(row[4]).ljust(20)}{str(row[5]).ljust(20)}{str(row[6]).ljust(20)}")

Söker efter titlar som matchar: game

ISBN           Titel                         Författare                    Genre                         Saldo BookStore     Saldo BokAffären    Saldo Affären med böcker
9780006174554  Patriot Games                 Tom Clancy                    Fiction                       4                   7                   0                   
9780007448036  A Game Of Thrones             George R.R. Martin            Fantasy, Fiction              9                   0                   0                   


LLMad kod, fick hjälp av en klasskompis med syntax som är mer interaktiv med användaren när man ska flytta böcker mellan butiker.

In [11]:
def visa_bocker():
    with engine.connect() as connection:
        query = """
        SELECT B.ISBN13, B.Titel, F.Förnamn + ' ' + F.Efternamn AS Författare, B.Pris
        FROM Böcker B
        JOIN BöckerOchFörfattare bof ON B.ISBN13 = bof.BokId
        JOIN Författare F ON bof.FörfattarID = F.ID
        ORDER BY B.Titel
        """
        df = pd.read_sql(query, connection)
        return df

# Funktion för att visa lagersaldo,
def visa_lagersaldo():
    with engine.connect() as connection:
        query = """
        SELECT L.ButiksID, B.Butiksnamn, L.ISBN, BK.Titel, L.Saldo
        FROM LagerSaldo L
        JOIN Butiker B ON L.ButiksID = B.ID
        JOIN Böcker BK ON L.ISBN = BK.ISBN13
        ORDER BY B.ID DESC
        """
        df = pd.read_sql(query, connection)
        return df

# Funktion för att visa butiker,
def visa_butiker():
    with engine.connect() as connection:
        query = "SELECT ID, Butiksnamn, Stad FROM Butiker ORDER BY Butiksnamn"
        df = pd.read_sql(query, connection)
        return df

# Funktion för att flytta böcker,
def flytta_bok(butik_fran, butik_till, isbn, antal):
    with engine.connect() as connection:
        try:
            connection.execute(text(f"EXEC FlyttaBok @FrånButik = {butik_fran}, @TillButik = {butik_till}, @ISBN = '{isbn}', @Antal = {antal}"))
            connection.commit()
            print(f"Flyttade {antal} exemplar av bok med ISBN {isbn} från butik {butik_fran} till butik {butik_till}")
            return True
        except Exception as e:
            print(f"Ett fel uppstod: {e}")
            return False

# Huvudprogram,
print("Tillgängliga butiker:")
butiker_df = visa_butiker()
display(butiker_df)

print("\nTillgängligt lagersaldo:")
lager_df = visa_lagersaldo()
display(lager_df)

# Be användaren om input,
butik_fran = int(input("Ange ButikID att flytta från: "))
butik_till = int(input("Ange ButikID att flytta till: "))
isbn = input("Ange ISBN13 för boken som ska flyttas: ")
antal = int(input("Ange antal böcker att flytta: "))

# Utför flytten,
if flytta_bok(butik_fran, butik_till, isbn, antal):
    # Visa uppdaterat lagersaldo
    print("\nUppdaterat lagersaldo:")
    ny_lager_df = visa_lagersaldo()
    display(ny_lager_df)

Tillgängliga butiker:


,ID,Butiksnamn,Stad
0,3,Affären med böcker,Malmö
1,2,BokAffären,Göteborg
2,1,Bookstore,Stockholm



Tillgängligt lagersaldo:


,ButiksID,Butiksnamn,ISBN,Titel,Saldo
0,3,Affären med böcker,9780440218807,Bravo Two Zero,0
1,3,Affären med böcker,9780552168984,Today Everything Changes,2
2,3,Affären med böcker,9780552171069,The Good Psychopath's Guide to Success,1
3,3,Affären med böcker,9780006174554,Patriot Games,0
4,3,Affären med böcker,9780425191187,Red Rabbit,1
5,3,Affären med böcker,9780425240335,The Hunt for Red October,3
6,3,Affären med böcker,9781408727836,Tom Clancy Red Winter,0
7,3,Affären med böcker,9780007448036,A Game Of Thrones,0
8,3,Affären med böcker,9780006479895,A Clash Of Kings,2
9,3,Affären med böcker,9780007447848,A Storm Of Swords,4


Flyttade 1 exemplar av bok med ISBN 9780440218807 från butik 1 till butik 2

Uppdaterat lagersaldo:


,ButiksID,Butiksnamn,ISBN,Titel,Saldo
0,3,Affären med böcker,9780440218807,Bravo Two Zero,0
1,3,Affären med böcker,9780552168984,Today Everything Changes,2
2,3,Affären med böcker,9780552171069,The Good Psychopath's Guide to Success,1
3,3,Affären med böcker,9780006174554,Patriot Games,0
4,3,Affären med böcker,9780425191187,Red Rabbit,1
5,3,Affären med böcker,9780425240335,The Hunt for Red October,3
6,3,Affären med böcker,9781408727836,Tom Clancy Red Winter,0
7,3,Affären med böcker,9780007448036,A Game Of Thrones,0
8,3,Affären med böcker,9780006479895,A Clash Of Kings,2
9,3,Affären med böcker,9780007447848,A Storm Of Swords,4
